## [作業重點]
確保你了解隨機森林模型中每個超參數的意義，並觀察調整超參數對結果的影響

## 作業

1. 試著調整 RandomForestClassifier(...) 中的參數，並觀察是否會改變結果？
2. 改用其他資料集 (boston, wine)，並與回歸模型與決策樹的結果進行比較

# 基本原理和結構:

https://www.jamleecute.com/random-forests-%E9%9A%A8%E6%A9%9F%E6%A3%AE%E6%9E%97/

https://rpubs.com/skydome20/R-Note16-Ensemble_Learning

所謂的隨機森林，就是運用 Bagging + CART決策樹，也就是說Model-1 ~ Model-n全都都是用決策樹來建模，而這麼多棵的樹組合在一起，所以才稱為「森林」。

要注意的是，隨機森林在抽樣過程中，不只是對 Row 進行抽樣，同時也會對 Column 抽樣，因此產生的子集資料，其實是對欄跟列抽樣後的結果。之後再針對這些子集資料，各自訓練一棵決策樹，形成隨機森林。

事實上，在面對資料中有共線性(collinearity)跟類別不平衡(Class Imbalance Problem)，而這些問題會對預測結果造成不良影響時(若是探討對「變數解釋性」的影響，則需要用 Lasso 或 Stepwise 來解決)，隨機森林是倍受青睞的演算法。

其概念應該不難理解：「對 Row 抽樣時，可以部份解決類別不平衡來影響預測的問題；對 Column 抽樣時，可以部份解決共線性來影響預測的問題」。

# 構建過程:

1. 從原始訓練集中使用Bootstraping方法隨機有放回採樣取出m個樣本，共進行n_tree次採樣。生成n_tree個訓練集。

2. 對n_tree個訓練集，我們分別訓練n_tree個決策樹模型。

3. 對於單個決策樹模型，假設訓練樣本特徵的個數為n，那麼每次分裂時根據信息增益/信息增益比/基尼指數，選擇最好的特徵進行分裂。

4. 每棵樹都已知這樣分裂下去，知道該節點的所有訓練樣例都屬於同一類。在決策樹的分裂過程中不需要剪枝。

5. 將生成的多顆決策樹組成隨機森林。對於分類問題，按照多棵樹分類器投票決定最終分類結果；對於回歸問題，由多顆樹預測值的均值決定最終預測結果。

https://kknews.cc/tech/5v4943l.html

# 1.)為甚麼要用RF?

    a.)若不對決策樹進行限制 (樹深度、葉子上至少要有多少樣本等)，決策樹非常容易過擬合

    b.)Bagging(Bootstrap Aggregation) Trees模型在演算法中納入了隨機的元素，有效的
       降低了單一樹模型的高變異性與提升模型預測正確率。然而在bagging中的trees並非所有
       都是彼此相互獨立的，因為在每一棵樹切分節點時都是考慮所有原始的預測變數。也因爲
       上述關係，來自不同bootstrapped samples的樹彼此的結構都會有些類似（尤其是在樹
       的上半部，用來切割的前幾大變數都會非常類似）。樹的結構相遇的這個特性就稱作tree 
       correlation，它阻礙了Bagging最適地降低預測目標值的變異(variance)。

    決策樹:
    https://www.jamleecute.com/decision-tree-cart-%e6%b1%ba%e7%ad%96%e6%a8%b9/

    Bagging(Bootstrap Aggregation):
    https://www.jamleecute.com/regression-tree-%E8%BF%B4%E6%AD%B8%E6%A8%B9-bagging-bootstrap-aggrgation-r%E8%AA%9E%E8%A8%80/
       
       為了更近一步降低變異，我們需要最小化樹與樹之間的相關性。這可以透過注入更多的隨
       機性到長樹的過程中來做到。

       Random Forests是透過以下兩步驟來達成的：

          -Bootstrap (拔靴法) : 
           跟Bagging很類似，每一顆樹都是建立自不同的bootstrapped sample，讓他們稍稍
           不一樣並稍稍去相關性。

          -Split-variable randomization (變數切割的隨機性) : 
           每一次在執行變數切割時，搜尋切割變數的範圍被限縮為隨機的子集合，即隨機挑選
           m個隸屬於總p個變數的子集合作為切割搜尋變數的範圍。對回歸樹來說，預設使用
           m=p3，是個可經調教的參數。當m=p的時候，則跟只進行步驟1的結果一樣。

           因為每棵樹都是來自不同的隨機bootstrapped sample且每一次切割都是隨機挑選變
           數的子集合，因此樹與樹之間的關聯性會下降地較Bagging更低。


# 2.)集成學習是甚麼概念?(大哉問)

先來幾篇值得一看的資料:

http://violin-tao.blogspot.com/2018/01/ml-ensemble.html

https://medium.com/@chih.sheng.huang821/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-ensemble-learning%E4%B9%8Bbagging-boosting%E5%92%8Cadaboost-af031229ebc3

https://rpubs.com/skydome20/R-Note16-Ensemble_Learning


所謂的 Ensemble Learning ，其實就是「三個臭皮匠，勝過一個諸葛亮」的概念，希望藉由團隊合作，結合多種模型的表現，提升最後的預測/分類結果。一般有三種常見的架構：

    -Bagging (Bootstrap aggregating)

    -Boosting

    -Stacking (short for stacked generalization, also called meta ensembling)
    
這三種架構在結合模型時使用的概念並不一樣，以下依序介紹概念:
    

# a.) Bagging:

    演算法上，是將樣本重複抽樣(取後放回)，產生多個子資料集(Subsets)後，依序建立多個模型，最後再將所有模型的結 
    果彙整在一起。如果是預測問題(Regression Problem)，那就把所有結果平均起來(Average)；如果是分類問題
    (Classification Problem)，那就用投票法(Voting)，判斷哪個類別出現最多次。

    在模型 Bias 跟 Variance 的議題上，Bagging 的手法有助於降低 variance。

    當每個子集資料在建模的時候，每個模型若獨自拿出來看，會發現都是一個「強模型」(較複雜的模型)，具有低 bias 跟
    高 variance 的特性；而如今，我們把不同高 variance 的模型結合在一起後，因為是平均(投票)的概念，其結果就會
    趨近於整體的平均表現，因此 variance 就不會太大。
    
    Supplement:為何說Bagging是減少variance，Boosting是減少Bias?
    
    https://www.zhihu.com/question/26760839
    
    Example: Random Forest
    
    
# b.) Boosting:

    跟 Bagging 使用多個「強模型」不同， Boosting 會強調使用上需要多個「弱模型」才可以。

    其概念用「三個臭皮匠，勝過一個諸葛亮」來形容最為貼切，也就是需要有多個非常弱的「臭皮匠」，而這些臭皮匠彼此
    是互補的。也就是說： M2 模型要能辦到 M1 模型所辦不到的， M3 要辦到 M2 所辦不到的……。

    會強調要「弱模型」，是因為模型本身如果「有點強」還不行。今天 M1 M2 M3 如果太複雜(太強)，那彼此之間就會互相
    干擾，影響最後預測/分類結果；唯有彼此都是「弱模型」，才能好好專注在自己本身的預測/分類，然後再把彼此的成果
    結合一起，這就是 Boosting 的概念。(當然以計算效率的考量，這樣做也比較快)
    
    In boosting, why are the learners “weak”?
    
    https://stats.stackexchange.com/questions/23388/in-boosting-why-are-the-learners-weak

    在演算法中，要從資料中找 M1 M2 M3…的模型是有順序的，概念跟 Bagging完全不一樣：

        在 Bagging 時，我們是將資料做抽樣，因此獲得許多子集資料，並各別建模後，把結果平均/投票；在 Boosting 
        時，一開始先建一個簡單的模型 M1 ，此時會有預測錯誤的資料，把這些資料的權重加大，建 M2 模型，然後又會有
        預測錯的資料，再把資料的權重加大，建 M3 模型…

    在模型 Bias 跟 Variance 的議題上，Boosting 的手法有助於降低 bias。

    由於使用上是拿「弱模型」來用，這些弱模型其實是高 bias 跟 低 variance 的，並且每次迭代的時候，都會基於先前
    的模型上進行優化(用梯度下降法，決定這次模型建在哪裡能使損失函數下降最多)。既然是降低損失函數，表示過程中會
    越來越逼近實際值，換句話說，就是逐漸降低 bias 的意思。
    
    Boosting的概念有點太玄，第一次都會有點聽不太懂@@
    
    不過沒關係，有這篇! http://violin-tao.blogspot.com/2018/01/ml-ensemble.html 
    
    Supplement:為何說Bagging是減少variance，Boosting是減少Bias?
    
    https://www.zhihu.com/question/26760839
    
    Example:Gradient Boosting Machine(XGboost) (聽說是Kaggle神兵利器啊~~XDD)
    

# c.) Stacking:

    最後要討論的 emsemble 技巧是 stacking，其概念很簡單：

    「在訓練多個模型、得到多個預測值/分類結果後，與其使用投票法(hard voting)或平均法(average)將這些結果整合
    (ensemble)起來，為何不多訓練一個模型來做這樣的整合呢？」

    Stacking 的核心概念：

        今天已經訓練好三個機器學習的模型，分別是 linear regression, support vector regression 跟 CART 
        decision tree。當有一筆新資料需要預測時，會各自得到三個預測值(y1, y2, y3)，然後接下來作為最終模型(又稱meta-model, blender,  
        meta learner)的輸入值，得到最終預測結果(y.final)。
    
    這跟傳統上的做法不太一樣，我們直覺上會把(y1, y2, y3)的結果直接拿來平均(預測問題)或投票法(分類問題)，得到
    最後的結果；不過 Stacking 採用另一個模型(blender)來取代這樣的概念。換句話說，也就是「把本來的預測結果再進
    一步做預測」的感覺。
    
    因此，Stacking 的演算法可以分成兩個階段，應該不難理解：

       -Stacking：先訓練多個初始模型，其預測結果叫做 Meta-Data，作為最終模型(Meta-Model; Blender)的的輸入。

       -Blending：最終模型會取得 Meta-Data ，整合出最後結果(Predicted Results)。

   
    聽起來很簡單，不過當開始使用 Stacking 的技巧來建模時，想必會立刻遇到瓶頸：「咦？要怎麼訓練初始模型跟最終模
    型？並得到所謂的 meta-data？」
    
    基本上，在訓練初始模型取得 Meta-Data 時，會搭配類似 K-folds 的技巧，將訓練資料切割成數個子集資料
    (Subsets)。
   
    有點複雜?沒關係!有這篇:https://rpubs.com/skydome20/R-Note16-Ensemble_Learning
    
    


# 3.)OOB是啥?


這篇好懂! http://violin-tao.blogspot.com/2018/01/ml-ensemble.html


正是因為RF中每一顆樹所使用的樣本皆不一樣(都是隨機抽取部分)，使得直接使用其他未被用於建樹的樣本來做誤差驗證成為可能，這就是out-of-bag(OOB)的概念。

和Bagging一樣，bootstrap resample法的一個天然好處，就是隨機森林模型可以透過out-of-bag(OOB)的樣本誤差來作為有效與合理近似實驗誤差(test error)，也就不需要額外產生或犧牲訓練資料集了，OOB sample可供作為一個內建的驗證子集合。

OOB sample 的存在讓尋找使模型錯誤率趨於穩定的最適樹模型數量(ntree)更有效率；但是OOB error仍有幾項缺點:

    -和test error終究還是預期會不太相同

    -很多package都沒有可以追蹤在某一棵樹模型中，哪些是OOB sample哪些不是的功能。這樣在比須比較多個模型的成效 
     時，想要使用相同的驗證資料集來幫每個模型打分數是不可行的

    -技術上雖然可以對OOB sample計算特定指標(metrics)如root mean squared logarithmic error (RMSLE)，但並非 
     所有package都有內建這樣的運算功能

所以如果你想要比較多個模型的成效或是使用較不傳統的損失函數指標，你可能還是會選擇cross validation。



# 4.)RF的優缺點分別是?

優點:

    1.由於採用了集成算法，本身精度比大多數單個算法要好，所以準確性高。
    2.在測試集上表現良好，由於兩個隨機性的引入，使得隨機森林不容易陷入過擬合（樣本隨
      機，特徵隨機）。
    3.在工業上，由於兩個隨機性的引入，使得隨機森林具有一定的抗噪聲能力，對比其他算法具
      有一定優勢。
    4.由於樹的組合，使得隨機森林可以處理非線性數據，本身屬於非線性分類（擬合）模型。
    5.它能夠處理很高維度（feature很多）的數據，並且不用做特徵選擇，對數據集的適應能
       力強：既能處理離散型數據，也能處理連續型數據，數據集無需規範化。
    6.訓練速度快，可以運用在大規模數據集上。
    7.可以處理預設值（單獨作為一類），不用額外處理。
    8.由於有袋外數據（OOB），可以在模型生成過程中取得真實誤差的無偏估計，且不損失訓練
      數據量。
    9.在訓練過程中，能夠檢測到feature間的互相影響，且可以得出feature的重要性，具有一
      定參考意義。
    10.由於每棵樹可以獨立、同時生成，容易做成並行化方法。
    11.由於實現簡單、精度高、抗過擬合能力強，當面對非線性數據時，適於作為基準模型。

    
缺點:

    1.當隨機森林中的決策樹個數很多時，訓練時需要的空間和時間會比較大。

    2.隨機森林中還有許多不好解釋的地方，有點算是黑盒模型。

    3.在某些噪音比較大的樣本集上，RF的模型容易陷入過擬合。

# Coding:

In [1]:
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [7]:
# 讀取資料集
wine = datasets.load_wine()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=4)

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 4)
clf = RandomForestClassifier(n_estimators=20, max_depth=4, oob_score=True)

# 訓練模型
clf.fit(x_train, y_train)

# 預測測試集
y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  1.0


In [10]:
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [4]:
print(wine.feature_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [5]:
print("Feature importance: ", clf.feature_importances_)

Feature importance:  [0.08124482 0.02190514 0.01356202 0.02810382 0.03131733 0.02335048
 0.12873231 0.00675089 0.04080841 0.23694649 0.06877885 0.15271294
 0.16578651]


# Tuning the RF through Grid Search:

https://www.itdaan.com/tw/f04950dc8b0cdcebcd4854b78545010f

https://scikit-learn.org/stable/modules/grid_search.html

In [3]:
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

wine = datasets.load_wine()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=4)

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 4)
clf = RandomForestClassifier()


In [4]:
from sklearn.model_selection import GridSearchCV

#搜索兩種網格，一種是n_estimators，一種是max_features
param_grid = [
{'n_estimators': [3, 10, 30],
 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False],
 'n_estimators': [3, 10],
 'max_features': [2, 3, 4]},
]


sklearn 根據param_grid的值，首先會評估3×4=12種n_estimators和max_features的組合方式，接下來在會在bootstrap=False的情況下（默認該值為True），評估2×3=6種12種n_estimators和max_features的組合方式，所以最終會有12+6=18種不同的超參數組合方式。


In [6]:
grid_search = GridSearchCV(clf, param_grid, cv=5,
                          scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

每一種組合方式要在訓練集上訓練5次， 所以一共要訓練18×5=90 次，當訓練結束后，你可以通過best_params_獲得最好的組合方式。

In [7]:
#最好的組合方式:
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [8]:
#最好的模型:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
# 訓練模型
grid_search.best_estimator_.fit(x_train, y_train)

# 預測測試集
y_pred = grid_search.best_estimator_.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  1.0
